Planificación usando Pyomo - Parte 4
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/workshop-asocio-2018/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/workshop-asocio-2018/tree/master/) para explorar el repositorio usando `nbviewer`. 

### Definición del problema

Se desea realizar el planeamiento de la operacion de un sistema hidrotermico conformado por dos plantas hidraulica y dos termicas. La informacion es la siguiente:

 
 
* **Planificacion:**    4 etapas. 


* **Planta hidraulica 1:**    
    * Vol. maximo (V*) = 100.
    * Caudal max. turbinado (Q*) = 50.
    * Factor conversion ($\rho$) = 1.
    * Aporte por etapa ($A_i$) =  {21, 15, 12, 18}.
    * Volumen inicial ($V_o$) = 75.  
    
    
* **Planta hidraulica 2:**    
    * Vol. maximo (V*) = 100.
    * Caudal max. turbinado (Q*) = 45.
    * Factor conversion ($\rho$) = 1.
    * Aporte por etapa ($A_i$)  = {14, 8, 5, 11}.
    * Volumen inicial ($V_o$) = 75.  
    
    
* **Planta termica 1 :**    
    * Generacion maxima (G*) = 45
    * Costo combustible (CC) = 15. 
    
    
* **Planta termica 2 :**    
    * Generacion maxima (G*) = 40
    * Costo combustible (CC) = 15. 
 
 
* **Racionamiento:**   
    * Costo racionamiento (CR) = 1000 para todas las etapas. 
 
 
* **Demanda:**  80 para todas las etapas 
 

### Inicialización de Pyomo

In [1]:
from pyomo.environ import *

## Creación del modelo abstracto
model = AbstractModel()

### Formulación del modelo abstracto

In [3]:
## Datos

# Numero de etapas
model.P = Set(ordered=True)                      
                     
# Numero de plantas hidro
model.HD = Set(ordered=True)                     

# Numero de plantas termicas 
model.TM = Set(ordered=True)                     

# Costo de Racionamiento indexado por etapa
model.CR = Param(model.P)                         

# Costo de Combusstible indexado por termica
model.CC = Param(model.TM)                       

# Volumen inicial del embalse indexado por hidro
model.Vo = Param(model.HD)                         

# Demanda indexada etapa
model.D = Param(model.P)                         
         
# Aportes Hidrologicos indexado por plantas hidro
model.A = Param(model.HD,model.P)                

# Caudal turbinado indexado por plantas hidro
model.QH = Param(model.HD)                       

# Generacion termica maxima indexado por termicas
model.GM = Param(model.TM)                       

# La energia racionada indexada por etapa
model.R = Var(model.P, within=NonNegativeReals)    

# La generacion termica varia por etapa y por planta
model.G = Var(model.P, model.TM, within=NonNegativeReals)   

# El caudar turbinado varia por etapa y por plantas hidroelectricas
model.Q = Var(model.P, model.HD, within=NonNegativeReals)         

# El volumen final varia por etapa y por la planta hidro en que se encuentre
model.V = Var(model.P, model.HD, within=NonNegativeReals)

# El volumen vertido varia por etapa y por planta hidro
model.S = Var(model.P, model.HD, within=NonNegativeReals)

### Funcion objetivo

La funcion objetivo busca minimizar el costo total del racionamiento mas el costo total de la generacion termica:


$$\text{minimize} ~ \sum_{p=1}^{P} ~\Bigg\{ CR * DEF_p + \sum_{t=1}^T CC_t * GT_{p,t} ~\Bigg\}$$







In [4]:
## Definicion de la funcion objetivo

def obj_rule(model):
    return (sum(model.CR[i]*model.R[i] for i in model.P) + 
           sum(model.CC[j]*model.G[i,j] for j in model.TM for i in model.P))

model.obj = Objective(rule = obj_rule)

### Demanda

Por cada periodo de tiempo $p$ debe cumplirse que:

$$ DEF_p + \Bigg\{ \sum_{h=1}^H \rho_h * Q_{hp} \Bigg\} + \Bigg\{ \sum_{t=1}^T GT_{t p} \Bigg\} = dem_p$$


In [5]:
def demanda_rule(model,p):
        return (model.R[p] + 
                sum(model.G[p,j] for j in model.TM) + 
                sum(model.Q[p,j] for j in model.HD)== model.D[p])

model.demanda = Constraint(model.P,rule=demanda_rule)

### Continuidad de los volumenes de los embalses

Para el primer periodo, el embalse inicial $Vol_{h,0}$ es conocido, entonces:

$$Vol_{h,1} +Q_{h,1}+Ver_{h,1}=A_{h,1}+ Vol_{h,0}$$

In [6]:
def continuidad_rule(model,p, hd):
        if model.P[p] == 1 :
            return model.V[p,hd] + model.Q[p,hd] + model.S[p,hd] == model.Vo[hd]+ model.A[hd,1]     
        else:
            return -model.V[p-1,hd] + model.V[p,hd] + model.Q[p,hd] + model.S[p,hd] ==  model.A[hd,p]    

model.continuidad = Constraint(model.P, model.HD, rule=continuidad_rule)

### Caudal turbinado maximo

In [7]:
%%writefile -a model.py

def qmax_rule(model, p, HD):
    
    return model.Q[p, HD] <= model.QH[HD]

model.qmax = Constraint(model.P, model.HD, rule = qmax_rule)

Writing model.py


### Generacion térmica maxima

In [8]:
def gtmax_rule(model,p,TM):
    return model.G[p,TM] <= model.GM[TM]

model.gtmax = Constraint(model.P, model.TM, rule=gtmax_rule)

### Manejo de datos usando DataObjects

In [9]:
## crea el objeto para cargar los datos
data = DataPortal()

In [ ]:
### set P  := 1 2 3 4 ;

In [10]:
%%writefile P.csv
P 
1 
2 
3 
4    

Overwriting P.csv


In [11]:
data.load(filename='P.csv', set=model.P, format='set')
data['P']

[1, 2, 3, 4]

In [ ]:
### set HD := 1 2 ;

In [12]:
%%writefile HD.csv
HD
1
2

Overwriting HD.csv


In [13]:
data.load(filename='HD.csv', set=model.HD, format='set')
data['HD']

[1, 2]

In [14]:
### set TM := 1 2 ;

In [15]:
%%writefile TM.csv
TM
1
2

Overwriting TM.csv


In [16]:
data.load(filename='TM.csv', set=model.TM, format='set')
data['TM']

[1, 2]

In [17]:
### param CR := 
### 1 1000
### 2 1000
### 3 1000
### 4 1000 ;

In [18]:
%%writefile CR.csv
P, CR 
1, 1000
2, 1000
3, 1000
4, 1000

Overwriting CR.csv


In [19]:
data.load(filename='CR.csv', param=model.CR, index=model.P)
data['CR']

{1: 1000, 2: 1000, 3: 1000, 4: 1000}

In [20]:
## param CC := 
## 1 15
## 2 15 ;

In [21]:
%%writefile CC.csv
P, CC
1, 15
2, 15

Overwriting CC.csv


In [22]:
data.load(filename='CC.csv', param=model.CC, index=model.P)
data['CC']

{1: 15, 2: 15}

In [23]:
## param Vo :=
## 1 75
## 2 75 ;

In [24]:
%%writefile Vo.csv
HD, Vo
1, 75
2, 75

Overwriting Vo.csv


In [25]:
data.load(filename='Vo.csv', param=model.Vo, index=model.HD)
data['Vo']

{1: 75, 2: 75}

In [26]:
## param D :=
## 1 80
## 2 80
## 3 80
## 4 80 ;

In [27]:
%%writefile D.csv
P, D
1, 80
2, 80
3, 80
4, 80

Overwriting D.csv


In [28]:
data.load(filename='D.csv', param=model.D, index=model.P)
data['D']

{1: 80, 2: 80, 3: 80, 4: 80}

In [29]:
## param A :=
## 1 1 21
## 2 1 14
## 1 2 15
## 2 2 8
## 1 3 12
## 2 3 5
## 1 4 18
## 2 4 11 ;

In [30]:
%%writefile A.csv
HD, P, A
1,  1, 21
2,  1, 14
1,  2, 15
2,  2, 8
1,  3, 12
2,  3, 5
1,  4, 18
2,  4, 11 

Overwriting A.csv


In [31]:
data.load(filename='A.csv', param=model.A, index=(model.HD, model.P))
data['A']

{(1, 1): 21,
 (1, 2): 15,
 (1, 3): 12,
 (1, 4): 18,
 (2, 1): 14,
 (2, 2): 8,
 (2, 3): 5,
 (2, 4): 11}

In [32]:
## param QH := 
## 1 50 
## 2 45 ;

In [37]:
%%writefile QH.csv
HD, QH
1,  50 
2,  45

Writing QH.csv


In [38]:
data.load(filename='QH.csv', param=model.QH, index=model.HD)
data['QH']

{1: 50, 2: 45}

In [33]:
## param GM := 
## 1 45
## 2 40 ;

In [39]:
%%writefile GM.csv
TM, GM
1,  45
2,  40 

Writing GM.csv


In [40]:
data.load(filename='GM.csv', param=model.GM, index=model.TM)
data['GM']

{1: 45, 2: 40}

# Solucion

In [43]:

from pyomo.opt import SolverFactory

opt = SolverFactory('glpk')

instance = model.create_instance(data)

results = opt.solve(instance)
results

{'Problem': [{'Name': 'unknown', 'Lower bound': 990.0, 'Upper bound': 990.0, 'Number of objectives': 1, 'Number of constraints': 21, 'Number of variables': 37, 'Number of nonzeros': 59, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.0139007568359375}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [46]:
instance.display()

Model unknown

  Variables:
    R : Size=4, Index=P
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :   0.0 :  None : False : False : NonNegativeReals
          2 :     0 :   0.0 :  None : False : False : NonNegativeReals
          3 :     0 :   0.0 :  None : False : False : NonNegativeReals
          4 :     0 :   0.0 :  None : False : False : NonNegativeReals
    G : Size=8, Index=G_index
        Key    : Lower : Value : Upper : Fixed : Stale : Domain
        (1, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (1, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (2, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (2, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (3, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
        (3, 2) :     0 :  15.0 :  None : False : False : NonNegativeReals
        (4, 1) :     0 :  45.0 :  None : False : False : NonNegat

Planificación usando Pyomo - Parte 4
===

**Juan David Velásquez Henao**  
jdvelasq@unal.edu.co   
Universidad Nacional de Colombia, Sede Medellín  
Facultad de Minas  
Medellín, Colombia

---

Haga click [aquí](https://github.com/jdvelasq/workshop-asocio-2018/tree/master/) para acceder al repositorio online.

Haga click [aquí](http://nbviewer.jupyter.org/github/jdvelasq/workshop-asocio-2018/tree/master/) para explorar el repositorio usando `nbviewer`. 